In [60]:
# Process the data
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from process_data import TextPreprocess


radonpy_data = pd.read_csv('./radonpy.csv')
# Remove the space of column name
radonpy_data.columns = radonpy_data.columns.str.replace(' ', '')

smiles_list = radonpy_data['smiles'].tolist()

smiles_process = TextPreprocess()
word2index, index2word, word_count = smiles_process.create_vocabulary(smiles_list)



In [65]:
from generative_model import VAE_Encoder, VAE_Decoder, VAE

word_count = 39
input_dim = 64
latent_dim = 32
nhead=1
hidden_dim = 128
output_dim = 39
number_layer = 1

encoder = VAE_Encoder(word_count=word_count, 
                      d_model=input_dim, 
                      latent_dim=latent_dim, 
                      nhead=nhead)

decoder = VAE_Decoder(latent_dim=latent_dim,
                      d_model=input_dim,
                      hidden_dim=hidden_dim,
                      output_dim=output_dim,
                      num_layers=number_layer)

In [66]:
x = smiles_process.text_to_index(smiles_list, padding=True)
x = torch.tensor(x, dtype=torch.int64)  # Convert to torch.long data type
# Only use the first 10 samples
x = x[:120]
x.shape

torch.Size([120, 164])

In [67]:
vae = VAE(encoder, decoder)

z, mu, logvar = vae.encoder(x)
print('z shape:', z.shape)
print('mu shape:', mu.shape)
print('logvar shape:', logvar.shape)

z shape: torch.Size([120, 164, 32])
mu shape: torch.Size([120, 164, 32])
logvar shape: torch.Size([120, 164, 32])


In [68]:
decoder

VAE_Decoder(
  (decoder): RNN(64, 128, batch_first=True)
  (fc): Linear(in_features=128, out_features=39, bias=True)
  (softmax): Softmax(dim=1)
)

In [74]:
decoder_x = torch.randn(120, 164, 64)
hidden = torch.randn(120, 164, 32)

decoder_out, decoder_hidden = vae.decoder(decoder_x, z)
print('decoder_out shape:', decoder_out.shape)
print('decoder_hidden shape:', decoder_hidden.shape)

After linear transformation, the hidden shape is:  torch.Size([1, 120, 128])
decoder_out shape: torch.Size([120, 164, 39])
decoder_hidden shape: torch.Size([1, 120, 128])


In [ ]:
# Train the model 
from training import Data_loader, Training

def loss_function(decoder_out, x):
    criterion = nn.CrossEntropyLoss()
    decoder_out = decoder_out.view(-1, 164)
    # Convert x to float data type
    x = x.float()
    loss = criterion(decoder_out, x)
    return loss
    

optimizer = torch.optim.Adam(vae.parameters(), lr=0.001)

# Traing the model
for epoch in range(10):
    optimizer.zero_grad()
    z, mu, logvar = vae.encoder(x)
    decoder_out, decoder_hidden = vae.decoder(decoder_x, z)
    print('decoder_out shape:', decoder_out.shape)
    loss = loss_function(decoder_out, x)
    loss.backward()
    optimizer.step()
    print('Epoch:', epoch, 'Loss:', loss.item())



In [75]:
decoder_out.shape, x.shape


(torch.Size([120, 164, 39]), torch.Size([120, 164]))

In [76]:
z, mu, logvar = vae.encoder(x)
decoder_out, decoder_hidden = vae.decoder(decoder_x, z)
decoder_out.shape, decoder_hidden.shape

After linear transformation, the hidden shape is:  torch.Size([1, 120, 128])


(torch.Size([120, 164, 39]), torch.Size([1, 120, 128]))

In [ ]:
z, mu, logvar, decoder_out = vae(x, decoder_x)
decoder_out.shape

In [57]:
class mlp(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(mlp, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.softmax(x)
        return x
    
x = torch.randn(10, 5)
y = torch.tensor([0, 1, 1, 1, 0, 0, 1, 0, 1, 1],dtype=torch.int64)
model = mlp(5, 3, 6)
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

pred = model(x)

loss_value = loss(pred, y)
loss_value

tensor(1.8215, grad_fn=<NllLossBackward0>)

In [101]:
batch_size = 3
seq_len = 8
class_num = 5

y = torch.tensor([[0, 1, 2, 3, 4, 0, 1, 2], [0, 1, 2, 3, 4, 0, 1, 2], [0, 1, 2, 3, 4, 0, 1, 2]], dtype=torch.int64)
x = torch.randn(batch_size, seq_len, class_num)
x = torch.softmax(x, dim=2)
print('x shape:', x.shape)
print('y shape:', y.shape)

x shape: torch.Size([3, 8, 5])
y shape: torch.Size([3, 8])


In [102]:
x

tensor([[[0.0717, 0.5718, 0.2268, 0.0802, 0.0496],
         [0.0244, 0.0448, 0.0856, 0.3895, 0.4556],
         [0.2146, 0.0642, 0.3530, 0.2780, 0.0901],
         [0.1565, 0.3633, 0.1562, 0.2299, 0.0942],
         [0.0460, 0.3356, 0.1258, 0.3278, 0.1646],
         [0.1453, 0.3395, 0.0956, 0.3170, 0.1027],
         [0.1965, 0.0658, 0.0807, 0.1774, 0.4797],
         [0.2449, 0.3520, 0.0967, 0.1083, 0.1980]],

        [[0.1276, 0.6744, 0.0333, 0.0371, 0.1276],
         [0.0917, 0.4395, 0.1376, 0.2452, 0.0860],
         [0.1289, 0.0884, 0.4184, 0.0604, 0.3039],
         [0.2570, 0.1424, 0.1081, 0.2276, 0.2649],
         [0.2934, 0.2852, 0.2117, 0.0753, 0.1345],
         [0.3455, 0.0548, 0.1700, 0.3496, 0.0800],
         [0.1288, 0.7209, 0.0719, 0.0504, 0.0280],
         [0.2694, 0.1529, 0.4755, 0.0610, 0.0412]],

        [[0.6993, 0.0538, 0.0968, 0.0697, 0.0804],
         [0.0193, 0.2882, 0.1796, 0.2169, 0.2960],
         [0.2671, 0.0363, 0.0976, 0.1645, 0.4346],
         [0.1713, 0.5003, 0